In [ ]:
import numpy as np
import sys
import os
from dataclasses import dataclass
from scipy.optimize import least_squares, fmin
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from typing import Dict, List
import time
from numpy.typing import ArrayLike
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.loading_util import make_path
from Functions.py_functions.tire_model.tire_model_pacejka_2010 import *
from Functions.py_functions.tire_model.tire_model_fitting import *
from Functions.py_functions.tire_model.ttc_loader import *
from Functions.py_functions.tire_model.tire_fitting_masks import *
from Functions.py_functions.tire_model.tire_model_utils import *
from Functions.py_functions.constants import *

In [ ]:
# Get the ttc data for the tire you want to fit
combi_runs, cornering, drive_brake, name = load_runs(get_R20_18x6_7_runs())
# create all the boring lists and stuff
params_list: List = []
error_list: List = []

In [ ]:
# If you want to start from the default tir file
tire_model = readTIR(make_path('./Data/TTCData/TIR_Templates/FSAE_Defaults.tir'))
# tire_model = readTIR(make_path('./Data/TTCData/TIR_Templates/H_R20_18X6_7_V1.tir'))
# or from a existing set of parameters
# tire_model = tire_model_from_arr(H_LC0_18X6_7)

In [ ]:
tire_model.UNLOADED_RADIUS = 0.223
tire_model.NOMPRES = 83000
tire_model.FNOMIN = 1100

We should look into LMA as a alternative to the GNA that we are using now in the least squares fitting (see https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm)

In [ ]:
def parameter_estimation_function_dev(x1, run, tk: TireMFModel, params_list, error_list, loss, mask: ArrayLike, ts: bool = False):
    error = parameter_estimation_function(x1, run, tk, params_list, error_list, loss, mask, ts)
    # calculate rms error
    rms_error = np.sqrt(np.mean(np.square(error)))
    return rms_error

def run_fit_dev(tire_model, data, loss_func, mask, params_list, error_list, ts=False, graph=False, fmin_t=True):
    if fmin_t:
        sol = fmin(parameter_estimation_function_dev, tire_model.dump_params()[mask], args=(data, tire_model, params_list, error_list, loss_func, mask,), disp=True)
        out = parameter_estimation_function(sol, data, tire_model, params_list, error_list, loss_func, mask)
    else:
        sol = least_squares(parameter_estimation_function, tire_model.dump_params()[mask], args=(data, tire_model, params_list, error_list, loss_func, mask,), method='trf', jac='2-point', verbose=2, ftol=0.001)
        out = parameter_estimation_function(sol.x, data, tire_model, params_list, error_list, loss_func, mask)
    if graph:
        split_run_with_MF_SA(data, tire_model, f"After Fy, Fx, Mz, and Mx Optimization")

In [ ]:
def clean_press(dd):
    del_8 = dd.P - 8*PSI_TO_PA
    del_10 = dd.P - 10*PSI_TO_PA
    del_12 = dd.P - 12*PSI_TO_PA
    del_14 = dd.P - 14*PSI_TO_PA
    delta_press_abs = np.minimum(np.minimum(np.abs(del_8), np.abs(del_10)), np.minimum(np.abs(del_12), np.abs(del_14)))

    delta_press = np.zeros(dd.P.shape)
    delta_press[np.abs(del_8) == delta_press_abs] = del_8[np.abs(del_8) == delta_press_abs]
    delta_press[np.abs(del_10) == delta_press_abs] = del_10[np.abs(del_10) == delta_press_abs]
    delta_press[np.abs(del_12) == delta_press_abs] = del_12[np.abs(del_12) == delta_press_abs]
    delta_press[np.abs(del_14) == delta_press_abs] = del_14[np.abs(del_14) == delta_press_abs]
    return delta_press

In [ ]:
# filter the data to get only IA != 0, PHIT ~ 0, and SA not in a transient state (when it inverts)
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[np.abs(df.SL) == 0]
df = df[df.TSTC > 50]
# Fit Fy to combined cornering and drive brake data
mask_pure = ["PCY1", "PDY1", "PDY2", "PEY1", "PEY2", "PKY1", "PKY2", "PKY4", "PKY6", "PHY1", "PHY2", "PVY1", "PVY2"]
# mask_com = ["PCY1", "PDY1", "PDY2", "PEY1", "PEY2", "PKY1", "PKY2", "PKY4", "PKY6"]
mask = ["PDY3", "PEY3", "PEY4", "PEY5", "PKY3", "PKY5", "PKY7", "PVY3", "PVY4"]
# mask = ["PDY3", "PEY4", "PEY5", "PKY3", "PKY5", "PKY7", "PVY3", "PVY4"]
mask = np.concatenate((generate_mask(mask_pure), generate_mask(mask), FY_MASK_IA_PRESS, FY_MASK_PRESS))
# print(mask)
run_fit_dev(tire_model, df, fit_f_y, mask, params_list, error_list, fmin_t=False)

In [ ]:
# filter the data to get only IA = 0, PHIT ~ 0, and SA not in a transient state (when it inverts)
df = remove_time_gaps(filter_vel(combi_runs, 11.1, 0.1))
df = df[np.abs(df.SL) != 0]
df = df[df.TSTC > 50]
# Fit Fy to combined cornering and drive brake data
run_fit_dev(tire_model, df, fit_f_y, np.concatenate((generate_mask(["RBY4", "RVY3"]), FY_MASK_COM)), params_list, error_list, fmin_t=False)

In [ ]:
df = remove_time_gaps(filter_vel(combi_runs, 11.1, 0.1))
# df = df[np.abs(df.SL) == 0]
df = df[df.TSTC > 50]
err = parameter_estimation_function_dev(tire_model.dump_params()[FY_MASK_COM], df, tire_model, params_list, error_list, fit_f_y, FY_MASK_COM)
print(err)
# split_run_with_MF_SA(df, tire_model, f"After Fy, Fx, Mz, and Mx Optimization")
tire_model.PVY1 = 0
tire_model.PVY2 = 0
tire_model.PHY1 = 0
tire_model.PHY2 = 0
tire_model.PEY3 = 0
err = parameter_estimation_function_dev(tire_model.dump_params()[FY_MASK_COM], df, tire_model, params_list, error_list, fit_f_y, FY_MASK_COM)
print(err)
split_run_with_MF_SA(df, tire_model, f"After Fy, Fx, Mz, and Mx Optimization")

In [ ]:
# plot rms error vs SA and FZ
df = remove_time_gaps(filter_vel(combi_runs, 11.1, 0.1))
# df = df[np.abs(clean_press(df)) < 2000]
df = df[np.abs(df.SL) == 0]
err = parameter_estimation_function(tire_model.dump_params()[FY_MASK_COM], df, tire_model, params_list, error_list, fit_f_y_norm, FY_MASK_COM)
fig = go.Figure()
fig.add_trace(go.Scattergl(x=np.rad2deg(df.SA), y=err, customdata=df.P/1000, mode='markers', marker=dict(color=np.abs(clean_press(df)), colorscale='Viridis', size=1, showscale=True), hovertemplate="SA: %{x:.2f} deg<br>IA: %{marker.color:.2f} deg<br>Error: %{y:.2f} N<br>Press: %{customdata:.2f} kPa"))
fig.add_histogram(y=err, histnorm='probability density', name="err")
fig.update_layout(template="plotly_dark", title="Error (N) vs SA", xaxis_title="SA", yaxis_title="RMS Error")
fig.show()

In [ ]:
# Fit Fx to combined cornering and drive brake data
df = remove_time_gaps(filter_vel(combi_runs, 11.1, 0.1))
df = df[np.abs(df.SL) != 0]
df = filter_sa(df)
# df = df[df.TSTC > 50]
run_fit_dev(tire_model, df, fit_f_x, np.concatenate((FX_MASK_IND, FX_MASK_PRESS, FX_MASK_IA)), params_list, error_list, fmin_t=False)

In [ ]:
# Fit Fx to combined cornering and drive brake data
df = remove_time_gaps(filter_vel(combi_runs))
df = df[np.abs(df.SL) != 0] # the cornering runs arent useful here
# df = df[df.TSTC > 50]
run_fit_dev(tire_model, df, fit_f_x, FX_MASK_COM, params_list, error_list, fmin_t=False)

In [ ]:
# Fit Mz to combined cornering and drive brake data
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[np.abs(df.SL) == 0]
# df = df[df.TSTC > 50]
run_fit_dev(tire_model, df, fit_m_z, np.concatenate((MZ_MASK_IND, MZ_MASK_PRESS, MZ_MASK_IA)), params_list, error_list, fmin_t=False)

In [ ]:
# Fit Mz to combined cornering and drive brake data
df = remove_time_gaps(filter_vel(combi_runs))
df = df[np.abs(df.SL) != 0]
# df = df[df.TSTC > 50]
run_fit_dev(tire_model, df, fit_m_z, MZ_MASK_COM, params_list, error_list, fmin_t=False)

In [ ]:
# Generate graphs to check the fit
# split_run_with_MF_SR(drive_brake, tire_model, f"After Fy, Fx, Mz, and Mx Optimization")
# split_run_fit(drive_brake, tire_model, f"After Fy Optimization")
# split_run_with_MF_SA(cornering, tire_model, f"After Fy, Fx, Mz, and Mx Optimization")
# cornering.MZ = 0
split_run_fit(df, tire_model, f"After Fy Optimization")
# split_run_fit(drive_brake, tire_model, f"After Fy Optimization")


In [ ]:
# Generate graphs to check smoothness of the fit
sweep_SA(tire_model, "Slip Angle")
sweep_SR(tire_model, "Slip Ratio")

In [ ]:
# Dump the parameters to a List
dump_param(tire_model.dump_params())


In [ ]:
# Write out the parameters with their names and descriptions
tire_model_2 = readTIR(make_path('./Data/TTCData/TIR_Templates/FSAE_Defaults.tir'))
# tire_model_2 = readTIR(make_path('./Data/TTCData/TIR_Templates/H_R20_18X6_7_V1.tir'))
param = tire_model_2.dump_params()
params = tire_model.dump_params()
# news = params
# news[FY_TURNSLIP] = param[FY_TURNSLIP]
# dump_param(news)
for i in FY_MASK_COM:#np.sort(np.concatenate((FY_MASK_PRESS, FY_MASK_IND, FY_MASK_IA, FY_MASK_IA_PRESS))):
        print(f"{i}\t{params[i]:.4f}\t{NAMES[i]}\t{LABLES[i]}\tDefault:{param[i]:.4f}")
print(get_param_list(MZ_TURNSLIP))

In [ ]:
print(f"{tire_model.PPX1:.4f} {tire_model.PPX3:.4f}")
print(f"{tire_model.PPX2:.4f} {tire_model.PPX4:.4f}")

In [ ]:
# Write out the parameters to a file
writeTIR(make_path('./Data/TTCData/TIR_Templates/H_R20_18X6_7_V3.tir'), tire_model)

In [ ]:
# Generate graphs to check the parameter convergence
gen_param_optimization_graph(params_list, FY_TURNSLIP, error_list)